In [1]:
# import sys
# !{sys.executable} -m nltk.download('punkt')
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from math import log, sqrt
import numpy as np
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayushi.sharma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayushi.sharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option('display.max_colwidth', -1)

In [65]:
mails = pd.read_csv("spam.csv", encoding='latin-1')

In [66]:
mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives around here though",NaN,NaN,NaN


In [67]:
def preprocess(mails):
    mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)
    mails.rename(columns = {'v1': 'labels', 'v2': 'message'}, inplace = True)
    return mails

In [68]:
mails = preprocess(mails)
mails.head()
# mails['labels'].value_counts()

,labels,message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [69]:
mails['label'] = mails['labels'].map({'ham': 0, 'spam': 1})
mails.drop('labels', axis = 1, inplace = True)
mails.head()

,message,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives around here though",0


In [70]:
from sklearn.model_selection import train_test_split 
x = mails.message 
y = mails.label 
  
# dividing X, y into train and test data 
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0)

In [71]:
x_train.head()

872     I'll text you when I drop x off                                                                                                                   
831     Hi mate its RV did u hav a nice hol just a message 3 say hello coz havenÃ¥Ãt sent u 1 in ages started driving so stay off roads!RVx              
1273    network operator. The service is free. For T & C's visit 80488.biz                                                                                
3314    FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For terms & conditions, visit www.07781482378.com
4929    Hi, the SEXYCHAT girls are waiting for you to text them. Text now for a great night chatting. send STOP to stop this service                      
Name: message, dtype: object

In [72]:
y_train.head()

872     0
831     0
1273    1
3314    1
4929    1
Name: label, dtype: int64

In [73]:
training_data = pd.merge(x_train, y_train, left_index=True, right_index=True)
training_data.head()

,message,label
872,I'll text you when I drop x off,0
831,Hi mate its RV did u hav a nice hol just a message 3 say hello coz havenÃ¥Ãt sent u 1 in ages started driving so stay off roads!RVx,0
1273,network operator. The service is free. For T & C's visit 80488.biz,1
3314,"FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For terms & conditions, visit www.07781482378.com",1
4929,"Hi, the SEXYCHAT girls are waiting for you to text them. Text now for a great night chatting. send STOP to stop this service",1


In [74]:
test_data = pd.merge(x_test, y_test, left_index=True, right_index=True)
test_data.head()

,message,label
4456,Aight should I just plan to come up later tonight?,0
690,Was the farm open?,0
944,"I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones",0
3768,Was gr8 to see that message. So when r u leaving? Congrats dear. What school and wat r ur plans.,0
1189,In that case I guess I'll see you at campus lodge,0


In [75]:
training_data['label'].value_counts()

0    3629
1    550 
Name: label, dtype: int64

In [76]:
test_data['label'].value_counts()

0    1196
1    197 
Name: label, dtype: int64

In [100]:
def process(message, lower_case = True, stem = True, stop_words = True, gram = 1):
    if lower_case:
        message = message.lower()
    message = message.replace('[^\w\s]', '')
    
    words = word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    if stop_words:
        sw = stopwords.words('english')
        words = [word for word in words if word not in sw]
    if stem:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]
    return ' '.join(map(str, words))

def wordCount(messages):
    counts = dict()
    for message in messages:
        words = message.split()
        for word in words:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    return counts

def wordTotal(messages):
    count = 0
    for message in messages:
        words = message.split()
        count = count + len(words)
    return count

def calc_total_prob(data):
    count_dict = dict()
    total_msgs = data.groupby('label')['message'].agg(wordTotal)
    return total_msgs
    
def calc_cond_prob(data):
    count_dict = dict()
    labeled_mssgs = data.groupby('label')['message'].agg(list)
    for label, mssgs in labeled_mssgs.items():
        count_dict[label] = wordCount(mssgs)
    return count_dict

def train(data):
    total_prob = calc_total_prob(data)
    cond_prob = calc_cond_prob(data)

    for label, word_map in cond_prob.items():
        for k,v in word_map.items():
            word_map[k] = word_map[k]/total_prob[label]
    return cond_prob

def predict_for_sentence(prob_matrix, sentence, label):
    pSentence = 0
    for word in sentence.split():
        pWord = prob_matrix.get(label, {}).get(word,0)
        pSentence = pSentence + pWord
    return pSentence

def predict(prob_matrix, sentence):
    p0 = predict_for_sentence(prob_matrix, sentence, 0)
    p1 = predict_for_sentence(prob_matrix, sentence, 1)
    return 0 if p0 > p1 else 1

def test(test_df, prob_matrix):
    test_df['label_pred'] = test_df['message'].apply(lambda x : predict(prob_matrix, x))
    return test_df
    
def main():
    training_data_copy = training_data.copy()
    training_data_copy['message'] = training_data_copy['message'].apply(process)
    prob_matrix = train(training_data_copy)
#     x = predict(prob_matrix, "Aight should I just plan to come up later tonight?")
#     print(x)
    test_data['message'] = test_data['message'].apply(process)
    test_pred = test(test_data, prob_matrix)
    return test_pred

 
test_pred = main()
wrong = test_pred[test_pred['label'] != test_pred['label_pred']]
print(wrong.size,"/",test_pred.size)
print("Accuracy:", ((test_pred.size-wrong.size)/test_pred.size)*100)
wrong.head()


420 / 4179
Accuracy: 89.9497487437186


,message,label,label_pred
3587,were/ar free give otherwi nalla adi entey nattil kittum,0,1
3348,brand,0,1
39,hello saturday text see decid anyth tomo tri invit anyth,0,1
2961,\none nowher ikno doesdiscount shitinnit\,0,1
2653,sorri thought call lot time lil busy.i call noon..,0,1


In [79]:
test_data.head()

,message,label,label_pred
4456,aight plan come later tonight,0,0
690,farm open,0,0
944,sent score sopha secondari applic school think think appli research cost also contact joke ogunrind school one less expens one,0,0
3768,gr8 see messag leav congrat dear school wat plan,0,0
1189,case guess 'll see campu lodg,0,0


In [115]:
# Using scikit - using tf-idf, bigrams
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

mails_sci = mails.copy()
# Done same processing like in w/o libs approach
mails_sci['message'] = mails_sci['message']
# .apply(process)

# Feature extraction
count_vect = CountVectorizer(ngram_range=(1,2), stop_words='english') 
# print(count_vect.get_feature_names())
counts = count_vect.fit_transform(mails_sci['message'])
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)  

# train model
# For text classification problems, use Multinomial Naive Bayes Classifier 
x2_train, x2_test, y2_train, y2_test = train_test_split(counts, mails_sci['label'], test_size=0.1, random_state=69)
model = MultinomialNB().fit(x2_train, y2_train)  

y2_predicted = model.predict(x2_test)
print(np.mean(y2_predicted == y2_test))
print(confusion_matrix(y2_test, y2_predicted)) 
print('--------------')
accuracy_score(y2_test, y2_predicted)
print(classification_report(y2_test, y2_predicted))

0.9587813620071685
[[480   0]
 [ 23  55]]
--------------
              precision    recall  f1-score   support

           0       0.95      1.00      0.98       480
           1       1.00      0.71      0.83        78

    accuracy                           0.96       558
   macro avg       0.98      0.85      0.90       558
weighted avg       0.96      0.96      0.96       558

